In [18]:
import pandas as pd

## Séries históricas disponíveis em
## http://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/mercado-a-vista/series-historicas/
arquivo_bovespa = 'COTAHIST_A2023.TXT'


## Estrutura do arquivo disponível em 
## http://www.b3.com.br/data/files/33/67/B9/50/D84057102C784E47AC094EA8/SeriesHistoricas_Layout.pdf

tamanho_campos=[2,8,2,12,3,12,10,3,4,13,13,13,13,13,13,13,5,18,18,13,1,8,7,13,12,3]

dados_acoes=pd.read_fwf(arquivo_bovespa, widths=tamanho_campos, header=0)

In [19]:
## Nomear as colunas

dados_acoes.columns = [
"tipo_registro",
"data_pregao",
"cod_bdi",
"cod_negociacao",
"tipo_mercado",
"noma_empresa",
"especificacao_papel",
"prazo_dias_merc_termo",
"moeda_referencia",
"preco_abertura",
"preco_maximo",
"preco_minimo",
"preco_medio",
"preco_ultimo_negocio",
"preco_melhor_oferta_compra",
"preco_melhor_oferta_venda",
"numero_negocios",
"quantidade_papeis_negociados",
"volume_total_negociado",
"preco_exercicio",
"ìndicador_correcao_precos",
"data_vencimento" ,
"fator_cotacao",
"preco_exercicio_pontos",
"codigo_isin",
"num_distribuicao_papel"]

# Eliminar a última linha
linha=len(dados_acoes["data_pregao"])
dados_acoes=dados_acoes.drop(linha-1)

# Ajustar valores com virgula (dividir os valores dessas colunas por 100)
listaVirgula=[
"preco_abertura",
"preco_maximo",
"preco_minimo",
"preco_medio",
"preco_ultimo_negocio",
"preco_melhor_oferta_compra",
"preco_melhor_oferta_venda",
"volume_total_negociado",
"preco_exercicio",
"preco_exercicio_pontos"
]

for coluna in listaVirgula:
    dados_acoes[coluna]=[i/100. for i in dados_acoes[coluna]]

In [24]:
dados_acoes['preco_ultimo_negocio']

0         15.03
1          0.30
2          1.45
3          2.71
4          8.50
          ...  
251785     0.86
251786     0.99
251787     1.12
251788     0.03
251789     0.05
Name: preco_ultimo_negocio, Length: 251790, dtype: float64

In [ ]:
lista = ['INBR32','PETR4','','','','','','']


dados_acoes.loc[dados_acoes['cod_negociacao'].isin(lista)]



In [ ]:
## !pip install yfinance
! pip install investpy

In [ ]:
### bilioteca inv para buscar todas as ações do br


import yfinance as yf
import pandas as pd
import investpy as inv

br = inv.stocks.get_stocks(country='brazil')
br['symbol']


carteira = []

for a in br['symbol']:
    if len(a) <=5:
        carteira.append(a+'.SA')

carteira


In [ ]:
import pandas as pd
from pandas_market_calendars import get_calendar

# Create a calendar object for the B3 exchange
b3 = get_calendar('BMF')

# Set the date range to the last 2 business days
end_date = pd.Timestamp('today', tz='UTC')
start_date = end_date - pd.DateOffset(days=2)

# Get the business days for the date range
business_days = b3.schedule(start_date=start_date, end_date=end_date)

# The last business day is the latest date in the list
last_business_day = business_days[-1].date()

# The penultimate business day is the second latest date in the list
penultimate_business_day = business_days[-2].date()
        
print("Last business day: ", last_business_day)
print("Penultimate business day: ", penultimate_business_day)

In [17]:
import pandas as pd
import pandas_market_calendars as mcal
import requests
from datetime import date

hoje = date.today()
# Cria o calendário da B3
b3_calendar = mcal.get_calendar('B3')

# Obtém o último e o penúltimo dia útil
days = b3_calendar.valid_days(start_date="2021-01-01", end_date=hoje)
ult_dia = str(days[-1].date().year % 100).zfill(2) + str(days[-1].date().month).zfill(2) + str(days[-1].date().day).zfill(2)
penult_dia = str(days[-2].date().year % 100).zfill(2) + str(days[-2].date().month).zfill(2) + str(days[-2].date().day).zfill(2)
print(ult_dia)
url = requests.get(f'https://www.anbima.com.br/informacoes/merc-sec/arqs/ms{ult_dia}.txt')
url

230215


<Response [404]>

**Títulos Públicos**

In [ ]:
def busca_titulos_tesouro_direto():
  url = 'https://www.tesourotransparente.gov.br/ckan/dataset/df56aa42-484a-4a59-8184-7676580c81e3/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1/download/PrecoTaxaTesouroDireto.csv'
  df  = pd.read_csv(url, sep=';', decimal=',')
  df['Data Vencimento'] = pd.to_datetime(df['Data Vencimento'], dayfirst=True)
  df['Data Base']       = pd.to_datetime(df['Data Base'], dayfirst=True)
  multi_indice = pd.MultiIndex.from_frame(df.iloc[:, :3])
  df = df.set_index(multi_indice).iloc[: , 3:]  
  return df
tesouro = busca_titulos_tesouro_direto()
tesouro.sort_index(inplace=True)
selic2025 = tesouro.loc[('Tesouro IPCA+', '2055-05-15')]
selic2025

In [18]:
def busca_recompras_tesouro():
  url = "https://www.tesourotransparente.gov.br/ckan/dataset/f30db6e4-6123-416c-b094-be8dfc823601/resource/30c2b3f5-6edd-499a-8514-062bfda0f61a/download/RecomprasTesouroDireto.csv"
  df  = pd.read_csv(url, sep=';', decimal=',')
  df['Vencimento do Titulo'] = pd.to_datetime(df['Vencimento do Titulo'], dayfirst=True)
  df['Data Resgate']       = pd.to_datetime(df['Data Resgate'], dayfirst=True)
  multi_indice = pd.MultiIndex.from_frame(df.iloc[:, :3])
  df = df.set_index(multi_indice).iloc[: , 3:]  
  return df

tesouro1 = busca_recompras_tesouro()
tesouro1.sort_index(inplace=True)
tesouro1

Quantidade  \
Tipo Titulo                        Vencimento do Titulo Data Resgate               
Tesouro IGPM+ com Juros Semestrais 2005-07-01           2005-01-13          2.00   
                                                        2005-02-03          1.00   
                                                        2005-03-03          2.00   
                                                        2005-03-10          1.00   
                                                        2005-03-24          0.20   
...                                                                          ...   
Tesouro Selic                      2029-03-01           2023-02-02        231.75   
                                                        2023-02-03        317.07   
                                                        2023-02-06        354.26   
                                                        2023-02-07        238.33   
                                                        2023-02-08        363.60   

                                                                           Valor  
Tipo Titulo                        Vencimento do Titulo Data Resgate              
Tesouro IGPM+ com Juros Semestrais 2005-07-01           2005-01-13       3545.04  
                                                        2005-02-03       1783.97  
                                                        2005-03-03       3595.96  
                                                        2005-03-10       1803.81  
                                                        2005-03-24        364.14  
...                                                                          ...  
Tesouro Selic                      2029-03-01           2023-02-02    2927759.94  
                                                        2023-02-03    4007829.28  
                                                        2023-02-06    4480184.24  
                                                        2023-02-07    3015595.70  
                                                        2023-02-08    4603022.14  

[55894 rows x 2 columns]